# Pattern File Generator


## Initial setup
Import dependencies:

In [1]:
import numpy as np
import pandas as pd

### Pattern generator class

In [2]:
class PatternGenerator:
    null_word = "0v{:016x}".format(0x0)
    null_header_length = 6
    bx_per_packet = 18
    frames_per_bx = 9
    packet_size = bx_per_packet * frames_per_bx
    
    def __init__(self, links):
        assert not hasattr(links, 'strip') and (hasattr(links, '__getitem__') or hasattr(links, '__iter__')), "links must be an iterable object of integers."
        self.links = pd.DataFrame(columns = links)
        
    def generateHeader(self):
        self.header = [
            "Board x1",
            "Quad/Chan :        " + "              ".join(["q{:02d}c{:01d}".format(i//4,i%4) for i in self.links]),
            "Link :         " + "                ".join(["{:03d}".format(i) for i in self.links])
        ]
        return self.header
       
    def padAndSegmentData(self, data):
        data = np.array(data)
        num_packets = int(np.ceil(data.size/(self.packet_size - self.null_header_length)))
        for i in range(num_packets):
            while np.all(data[i*self.packet_size:i*self.packet_size+self.null_header_length] == self.null_word) == False:
                data = np.insert(data, i*packet_size, self.null_word)
        return data
    
    def buildLink(self, index, data, pad=True):
        if pad:
            data = self.padAndSegmentData(data)
        else:
            data = np.array(data)
        self.links[index] = data
        return self.links[index]
    
    def writeToFile(self, filename):
        self.links = self.links.fillna(self.null_word)
        header = self.generateHeader()
        values = self.links.values
        frames = ["Frame %04d : " % i + (" ").join(values[i]) for i in range(len(values))]
        pattern = header + frames
        f = open(filename, "w")
        for i in pattern:
            f.write(i + "\n")
        f.close()
        return header + frames

## Pattern generation

#### In this example, we will create a pattern where each link is a defined offset from the previous

Define your 'valid' word and the null word and some other constants:

In [3]:
valid_word = "1v{:016x}".format(0xaaaaaaaaaaaaaaaa)
null_word = "0v{:016x}".format(0x0)

bx_per_packet = 18
frames_per_bx = 9
packet_size = bx_per_packet * frames_per_bx

num_links = 18

null_header_length = 6

Define the offsets and the function to build the data on each link:

In [4]:
offset_size = 3

offsets = [offset_size * i for i in range(num_links)]

def buildLinkPacket(offset):
    packet = [null_word for i in range(null_header_length)]
    packet += [null_word for i in range(offset)]
    while len(packet) < packet_size:
        packet.append(valid_word)
    return packet


Generate the pattern using the class defined above:

In [5]:
pg = PatternGenerator(range(40,58))
for i, link in enumerate(pg.links.columns):
    pg.buildLink(link, buildLinkPacket(offsets[i]))

Write pattern to file:

In [6]:
pg.writeToFile("pattern.txt");

In [7]:
# from emputils import PatternGenerator as PG

In [8]:
import emputils.patterngenerator


In [9]:
dir(emputils)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'patterngenerator']